In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from preprocessing import parse_aug_fn, parse_fn

In [2]:
train_split, valid_split = ['train[:90%]', 'train[90%:]']

train_data, info = tfds.load("cifar10", split=train_split, with_info=True)
valid_data = tfds.load("cifar10", split=valid_split)
test_data = tfds.load("cifar10", split=tfds.Split.TEST)

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
batch_size = 64 
train_num = int(info.splits['train'].num_examples / 10) * 9  

train_data = train_data.shuffle(train_num) 
train_data = train_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
train_data = train_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

valid_data = valid_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
valid_data = valid_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

test_data = test_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

In [4]:
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(64, 3, activation='relu', kernel_initializer='glorot_uniform')(inputs)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(128, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(256, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(128, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(64, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10)(x)

model_1 = keras.Model(inputs, outputs, name='model-1')
model_1.summary()

Model: "model-1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 256)       295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 128)         295040    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 64)          7379

In [5]:
logs_dirs = 'lab6-logs'
model_cbk = keras.callbacks.TensorBoard(log_dir='lab6-logs')

model_dirs = logs_dirs + '/models'
os.makedirs(model_dirs, exist_ok=True)
save_model = tf.keras.callbacks.ModelCheckpoint(model_dirs + '/save.h5', 
                                                monitor='val_catrgorical_accuracy', 
                                                mode='max')

In [6]:
model_1.compile(keras.optimizers.Adam(), 
                loss=keras.losses.CategoricalCrossentropy(from_logits=True), 
                metrics=[keras.metrics.CategoricalAccuracy()])

In [7]:
model_1.fit(train_data,
            epochs=100, 
            validation_data=valid_data,
            callbacks=[model_cbk, save_model])

Epoch 1/100
704/704 [==============================] - 13s 10ms/step - loss: 2.0783 - categorical_accuracy: 0.2111 - val_loss: 1.8405 - val_categorical_accuracy: 0.3218
Epoch 2/100
704/704 [==============================] - 6s 9ms/step - loss: 1.8810 - categorical_accuracy: 0.2985 - val_loss: 1.5402 - val_categorical_accuracy: 0.4138
Epoch 3/100
704/704 [==============================] - 6s 9ms/step - loss: 1.7597 - categorical_accuracy: 0.3518 - val_loss: 1.4390 - val_categorical_accuracy: 0.4730
Epoch 4/100
704/704 [==============================] - 6s 9ms/step - loss: 1.6686 - categorical_accuracy: 0.3924 - val_loss: 1.4774 - val_categorical_accuracy: 0.4698
Epoch 5/100
704/704 [==============================] - 6s 9ms/step - loss: 1.5999 - categorical_accuracy: 0.4228 - val_loss: 1.2870 - val_categorical_accuracy: 0.5452
Epoch 6/100
704/704 [==============================] - 6s 9ms/step - loss: 1.5447 - categorical_accuracy: 0.4454 - val_loss: 1.2176 - val_categorical_accuracy: 0.5

704/704 [==============================] - 6s 9ms/step - loss: 0.9037 - categorical_accuracy: 0.6967 - val_loss: 0.6620 - val_categorical_accuracy: 0.7804
Epoch 50/100
704/704 [==============================] - 6s 9ms/step - loss: 0.9053 - categorical_accuracy: 0.6976 - val_loss: 0.6675 - val_categorical_accuracy: 0.7750
Epoch 51/100
704/704 [==============================] - 6s 9ms/step - loss: 0.9036 - categorical_accuracy: 0.6952 - val_loss: 0.6754 - val_categorical_accuracy: 0.7750
Epoch 52/100
704/704 [==============================] - 6s 9ms/step - loss: 0.8957 - categorical_accuracy: 0.6987 - val_loss: 0.6657 - val_categorical_accuracy: 0.7780
Epoch 53/100
704/704 [==============================] - 6s 9ms/step - loss: 0.8907 - categorical_accuracy: 0.7021 - val_loss: 0.6477 - val_categorical_accuracy: 0.7810
Epoch 54/100
704/704 [==============================] - 6s 9ms/step - loss: 0.8940 - categorical_accuracy: 0.6998 - val_loss: 0.6890 - val_categorical_accuracy: 0.7734
Epoch

Epoch 98/100
704/704 [==============================] - 6s 9ms/step - loss: 0.7830 - categorical_accuracy: 0.7414 - val_loss: 0.6465 - val_categorical_accuracy: 0.7926
Epoch 99/100
704/704 [==============================] - 7s 9ms/step - loss: 0.7665 - categorical_accuracy: 0.7476 - val_loss: 0.6336 - val_categorical_accuracy: 0.8004
Epoch 100/100
704/704 [==============================] - 6s 9ms/step - loss: 0.7581 - categorical_accuracy: 0.7483 - val_loss: 0.6281 - val_categorical_accuracy: 0.8038


In [9]:
class CustomConv2D(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides=(1, 1), padding="VALID", **kwargs):
        super(CustomConv2D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = (1, *strides, 1)
        self.padding = padding

    def build(self, input_shape):
        kernel_h, kernel_w = self.kernel_size
        input_dim = input_shape[-1]
        # 創建卷積的權重值(weights)
        self.w = self.add_weight(name='kernel', 
                                 shape=(kernel_h, kernel_w, input_dim, self.filters),
                                 initializer='glorot_uniform',  # 設定初始化方法
                                 trainable=True)  # 設定這個權重是否能夠訓練(更動)
        # 創建卷積的偏差值(bias)
        self.b = self.add_weight(name='bias', 
                                 shape=(self.filters,),
                                 initializer='zeros',  # 設定初始化方法
                                 trainable=True)  # 設定這個權重是否能夠訓練(更動)

    def call(self, inputs):
        x = tf.nn.conv2d(inputs, self.w, self.strides, padding=self.padding) # 卷積運算
        x = tf.nn.bias_add(x, self.b)  # 加上偏差值
        x = tf.nn.relu(x)  # 激活函數
        return x

In [11]:
class SaveModel(tf.keras.callbacks.Callback):
    def __init__(self, weights_file, monitor='loss', mode='min', save_weights_only=False):
        super(SaveModel, self).__init__()
        self.weights_file = weights_file
        self.monitor = monitor
        self.mode = mode
        self.save_weights_only = save_weights_only
        if mode == 'min':
            self.best = np.Inf
        else:
            self.best = -np.Inf
        
    def save_model(self):
        if self.save_weights_only:
            self.model.save_weights(self.weights_file)
        else:
            self.model.save(self.weights_file)

    def on_epoch_end(self, epoch, logs=None):
        monitor_value = logs.get(self.monitor)
        if self.mode == 'min' and monitor_value < self.best:
            self.save_model()
            self.best = monitor_value
        elif self.mode == 'max' and monitor_value > self.best:
            self.save_model()
            self.best = monitor_value

In [12]:
class CustomCategoricalAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='custom_catrgorical_accuracy', **kwargs):
        super(CustomCategoricalAccuracy, self).__init__(name=name, **kwargs)
        # 記錄正確預測的數量
        self.correct = self.add_weight('correct_numbers', initializer='zeros')
        # 記錄全部資料的量數
        self.total = self.add_weight('total_numbers', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # 輸入答案為One-hot編碼，所以取最大的數值為答案
        y_true = tf.argmax(y_true, axis=-1)
        # 取預測輸出最大的數值為預測結果
        y_pred = tf.argmax(y_pred, axis=-1)
        # 比較預測結果是否正確，正確會返回True(正確)，錯誤會返回False(錯誤)
        values = tf.equal(y_true, y_pred)
        # 轉為浮點數True(正確)=1.0，False(錯誤)=0.0
        values = tf.cast(values, tf.float32)
        # 將values所有數值相加就會等於正確預測的總數
        values_sum = tf.reduce_sum(values)
        # 計算這個Batch的資料數量
        num_values = tf.cast(tf.size(values), tf.float32)
        self.correct.assign_add(values_sum)  # 更新正確預測的總數
        self.total.assign_add(num_values)  # 更新資料量的總數

    def result(self):
        # 計算準確率
        return tf.math.divide_no_nan(self.correct, self.total)

    def reset_states(self):
        # 每一次Epoch結束後會重新初始化變數
        self.correct.assign(0.)
        self.total.assign(0.)

In [16]:
def custom_categorical_crossentropy(y_true, y_pred):
    # x = tf.reduce_mean(-tf.reduce_sum(y_true * tf.log(y_pred), reduction_indices=[1]))
    x = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    return x

In [13]:
inputs = keras.Input(shape=(32, 32, 3))
x = CustomConv2D(64, (3, 3))(inputs)
x = layers.MaxPool2D()(x)
x = CustomConv2D(128, (3, 3))(x)
x = CustomConv2D(256, (3, 3))(x)
x = CustomConv2D(128, (3, 3))(x)
x = CustomConv2D(64, (3, 3))(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10)(x)

model_2 = keras.Model(inputs, outputs, name='model-2')
model_2.summary()

Model: "model-2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
custom_conv2d (CustomConv2D) (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
custom_conv2d_1 (CustomConv2 (None, 13, 13, 128)       73856     
_________________________________________________________________
custom_conv2d_2 (CustomConv2 (None, 11, 11, 256)       295168    
_________________________________________________________________
custom_conv2d_3 (CustomConv2 (None, 9, 9, 128)         295040    
_________________________________________________________________
custom_conv2d_4 (CustomConv2 (None, 7, 7, 64)          7379

In [14]:
logs_dirs = 'lab6-logs'
model_cbk = keras.callbacks.TensorBoard(log_dir='lab6-logs')

model_dirs = logs_dirs + '/models'
os.makedirs(model_dirs, exist_ok=True)
custom_save_model = SaveModel(model_dirs + '/custom_save.h5', 
                              monitor='val_custom_catrgorical_accuracy', 
                              mode='max', 
                              save_weights_only=True)

In [17]:
model_2.compile(keras.optimizers.Adam(), 
           loss=custom_categorical_crossentropy, 
           metrics=[CustomCategoricalAccuracy()])

model_2.fit(train_data,
            epochs=100, 
            validation_data=valid_data,
            callbacks=[model_cbk, custom_save_model])

Epoch 1/100
  3/704 [..............................] - ETA: 57s - loss: 2.3281 - custom_catrgorical_accuracy: 0.0729 WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0091s vs `on_train_batch_end` time: 0.0244s). Check your callbacks.


704/704 [==============================] - ETA: 0s - loss: 2.0563 - custom_catrgorical_accuracy: 0.2248

c:\users\asus\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\metrics.py:254: UserWarning: Metric CustomCategoricalAccuracy implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  warnings.warn('Metric %s implements a `reset_states()` method; rename it '


704/704 [==============================] - 7s 9ms/step - loss: 2.0563 - custom_catrgorical_accuracy: 0.2248 - val_loss: 1.7011 - val_custom_catrgorical_accuracy: 0.3650
Epoch 2/100
704/704 [==============================] - 6s 9ms/step - loss: 1.8342 - custom_catrgorical_accuracy: 0.3223 - val_loss: 1.5328 - val_custom_catrgorical_accuracy: 0.4306
Epoch 3/100
704/704 [==============================] - 7s 9ms/step - loss: 1.7113 - custom_catrgorical_accuracy: 0.3745 - val_loss: 1.3793 - val_custom_catrgorical_accuracy: 0.5022
Epoch 4/100
704/704 [==============================] - 6s 9ms/step - loss: 1.6171 - custom_catrgorical_accuracy: 0.4143 - val_loss: 1.3197 - val_custom_catrgorical_accuracy: 0.5226
Epoch 5/100
704/704 [==============================] - 6s 9ms/step - loss: 1.5484 - custom_catrgorical_accuracy: 0.4462 - val_loss: 1.2037 - val_custom_catrgorical_accuracy: 0.5754
Epoch 6/100
704/704 [==============================] - 6s 9ms/step - loss: 1.4924 - custom_catrgorical_accu

704/704 [==============================] - 7s 10ms/step - loss: 0.9101 - custom_catrgorical_accuracy: 0.6948 - val_loss: 0.7135 - val_custom_catrgorical_accuracy: 0.7614
Epoch 47/100
704/704 [==============================] - 7s 10ms/step - loss: 0.9053 - custom_catrgorical_accuracy: 0.6970 - val_loss: 0.6864 - val_custom_catrgorical_accuracy: 0.7646
Epoch 48/100
704/704 [==============================] - 7s 10ms/step - loss: 0.8869 - custom_catrgorical_accuracy: 0.7033 - val_loss: 0.6916 - val_custom_catrgorical_accuracy: 0.7680
Epoch 49/100
704/704 [==============================] - 7s 9ms/step - loss: 0.8990 - custom_catrgorical_accuracy: 0.6999 - val_loss: 0.6909 - val_custom_catrgorical_accuracy: 0.7680
Epoch 50/100
704/704 [==============================] - 6s 9ms/step - loss: 0.8817 - custom_catrgorical_accuracy: 0.7042 - val_loss: 0.6920 - val_custom_catrgorical_accuracy: 0.7658
Epoch 51/100
704/704 [==============================] - 6s 9ms/step - loss: 0.8812 - custom_catrgori

704/704 [==============================] - 6s 9ms/step - loss: 0.7851 - custom_catrgorical_accuracy: 0.7383 - val_loss: 0.6538 - val_custom_catrgorical_accuracy: 0.7914
Epoch 92/100
704/704 [==============================] - 6s 9ms/step - loss: 0.7802 - custom_catrgorical_accuracy: 0.7420 - val_loss: 0.6424 - val_custom_catrgorical_accuracy: 0.7922
Epoch 93/100
704/704 [==============================] - 6s 9ms/step - loss: 0.7859 - custom_catrgorical_accuracy: 0.7432 - val_loss: 0.6430 - val_custom_catrgorical_accuracy: 0.7880
Epoch 94/100
704/704 [==============================] - 6s 9ms/step - loss: 0.7775 - custom_catrgorical_accuracy: 0.7420 - val_loss: 0.6223 - val_custom_catrgorical_accuracy: 0.7960A: 1s - loss: 0.7758 - custom_catrg
Epoch 95/100
704/704 [==============================] - 6s 9ms/step - loss: 0.7799 - custom_catrgorical_accuracy: 0.7408 - val_loss: 0.6233 - val_custom_catrgorical_accuracy: 0.7936
Epoch 96/100
704/704 [==============================] - 6s 9ms/step 